### קוד מבוא

In [98]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [99]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [100]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [101]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [102]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [103]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [104]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [105]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [106]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


### ביצוע

#### עיבוד מידע טבלאי של הדמוגרף

In [107]:
demo=up_load_df(r'\\FILE-SRV\Jtmt\projections_team\תכניות אסטרטגיות לתחבורה ירושלים\תחזיות 2050\עבודת דמוגרף\קבלת מידע\תחזיות_פלסטינים_ערבים','‏‏PalestiniansResults_forecast_zone')

In [108]:
col=['male_2020',
 'female_2020',
 'male_2025',
 'female_2025',
 'male_2030',
 'female_2030',
 'male_2035',
 'female_2035',
 'male_2040',
 'female_2040',
 'male_2045',
 'female_2045',
 'male_2050',
 'female_2050']

In [109]:
demo=demo.pivot_table(index='Proj_Area',aggfunc=sum)[col]

In [110]:
year=['2020','2025','2030','2035','2040','2045','2050']

In [111]:
for x in year:
    demo['pop_{}'.format(x)]= demo['female_{}'.format(x)]+demo['male_{}'.format(x)]

#### שכבת אזורי תחזית של דמוגרף

In [112]:
proj_zones=up_load_shp(r'K:\projections_team\תכניות אסטרטגיות לתחבורה ירושלים\תחזיות 2050\עבודת דמוגרף\קבלת מידע\אזורי_תחזיות_פלסטינים\SHP\proj_zones_pls.shp')

In [113]:
proj_zones['Proj_Area']=proj_zones['projection']

In [114]:
proj_zones=proj_zones[['Proj_Area', 'geometry']]

#### חיבור נתוני דמוגרף

In [115]:
pd.merge(proj_zones,demo,on='Proj_Area',how='right').query('geometry.isna()')

Proj_Area geometry  male_2020  female_2020  male_2025  female_2025  \
36         50     None   1,232.43     1,124.17   1,379.29     1,269.85   

    male_2030  female_2030  male_2035  female_2035  male_2040  female_2040  \
36   1,546.52     1,436.05   1,738.47     1,625.90   1,942.32     1,827.51   

    male_2045  female_2045  male_2050  female_2050  pop_2020  pop_2025  \
36   2,142.52     2,026.16   2,334.97     2,216.95  2,356.60  2,649.13   

    pop_2030  pop_2035  pop_2040  pop_2045  pop_2050  
36  2,982.57  3,364.37  3,769.84  4,168.67  4,551.91

 אנחנו מוכנים לוותר עליהם 
 זה ניקוז של כל הבדואיים שאין להם אזור תחזית

In [116]:
proj_zones=pd.merge(proj_zones,demo,on='Proj_Area',how='left')

#### שכבת אזורי תנועה

In [117]:
taz=up_load_shp(r'W:\Model Versions\4.0\TAZ\TAZ_V4_221212_Published.shp')

In [118]:
taz=taz.query('main_secto=="Palestinian"')[['Taz_num','geometry']]

#### שכבת בינוי

In [119]:
bld=up_load_shp(r'K:\projections_team\תכניות אסטרטגיות לתחבורה ירושלים\תחזיות 2050\עבודת דמוגרף\פיזור_אוכלוסייה_באזורי_תנועה_פלסטינאים\SHP\palestinian_bld_area_pcbs_220710.shp')

In [120]:
bld=bld.to_crs(2039)

In [121]:
bld=bld.dissolve()

In [122]:
bld=bld[['geometry']]

#### הצלבת בין בינוי לבין אזורי תחזית ותנועה

In [123]:
col=['Proj_Area',
 'geometry',
 'pop_2020',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050']

In [124]:
demo_taz_bld=bld.overlay(taz).overlay(proj_zones[col])

C:\Users\gidon\Anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:2196: UserWarning: `keep_geom_type=True` in overlay resulted in 4 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [125]:
demo_taz_bld['small_area']=demo_taz_bld.area

In [126]:
demo_taz_bld=demo_taz_bld.set_index('Proj_Area')

In [127]:
demo_taz_bld['proj_sum_area']=demo_taz_bld.groupby(by='Proj_Area').sum()[['small_area']]

In [128]:
demo_taz_bld['pre_from_proj']=demo_taz_bld['small_area']/demo_taz_bld['proj_sum_area']

In [129]:
pop_year=['pop_2020',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050']

In [130]:
for y in pop_year:
    demo_taz_bld['{}'.format(y)]=demo_taz_bld['{}'.format(y)]*demo_taz_bld['pre_from_proj']

בדיקה שלא הומצא כמות אנשים

In [131]:
demo_taz_bld[pop_year].sum().sum()-proj_zones[pop_year].sum().sum()

3.725290298461914e-09

#### סכום אנשים לפי אזורי תנועה

In [132]:
taz_demo_pls=demo_taz_bld.pivot_table(index='Taz_num',aggfunc=sum)

In [133]:
add_year=['2025', '2030', '2035', '2040', '2045', '2050']

In [134]:
x=2020
for y in add_year:
    taz_demo_pls['precent_add_pop_{}'.format(y)]=taz_demo_pls['pop_{}'.format(y)]/taz_demo_pls['pop_{}'.format(str(x))]
    taz_demo_pls['precent_add_pop_{}'.format(y)]=taz_demo_pls['precent_add_pop_{}'.format(y)].round(2)
    taz_demo_pls['precent_add_pop_{}'.format(y)].fillna(0)
    x+=5

#### ייצוא

In [135]:
col=['pop_2020',
 'precent_add_pop_2025',
 'precent_add_pop_2030',
 'precent_add_pop_2035',
 'precent_add_pop_2040',
 'precent_add_pop_2045',
 'precent_add_pop_2050']

In [136]:
taz_demo_pls[col].to_excel(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020\מרכיבי_תחזית\taz_demo_pls_2020_and_pre_growth_till_2050.xlsx')